<a href="https://colab.research.google.com/github/amtayl25/1d3d/blob/main/Convergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install required libraries (try-catch safe)
"""
NOTE: You can save the Docker image state after running this block so that you don't have to run it every time you start a new environment.
"""
import os, re

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# ipywidgets
try:
    import ipywidgets
except ImportError:
    !pip install ipywidgets

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# vtk
try:
    import vtk
except ImportError:
    !pip install vtk

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

# meshio
try:
    import meshio
except ImportError:
    !pip install meshio

# pyvista
try:
    import pyvista
except ImportError:
    !pip install pyvista

--2025-08-06 14:16:12--  https://fem-on-colab.github.io/releases/fenics-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4180 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.08K  --.-KB/s    in 0s      

2025-08-06 14:16:12 (71.9 MB/s) - ‘/tmp/fenics-install.sh’ saved [4180/4180]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/ec24876d/releases/pybi

In [ ]:
# @title Import required modules
import os
WD_PATH = "" # If you're using the default Colab runtime, this is just where the runtime is located
#DATA_FILE_PATH = os.path.join(WD_PATH, "aorta_test.vtk")

# Libraries
import copy
import datetime
import importlib
import json
import matplotlib.pyplot as plt
import meshio
import networkx as nx
import numpy as np
import scipy
import vtk

from dolfin import *
from graphnics import *
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import cKDTree
from vtk.util.numpy_support import vtk_to_numpy
from xii import *

/usr/local/lib/python3.11/dist-packages/block/__init__.py:15: UserWarning: The cbc.block repository has moved to https://github.com/blocknics/cbc.block
  warnings.warn('The cbc.block repository has moved to https://github.com/blocknics/cbc.block', UserWarning)


Missing HsMG for fract norm computing


In [ ]:
from argparse import RawDescriptionHelpFormatter
from petsc4py import PETSc
from ast import Constant
from dolfin import *
from xii import *
import numpy as np
import time

# @title Convergence for exact solution in Masri et al. 2024: steady state 3D-FEM, 1D-dG

conv_study3d = open("conv_study3d.txt", "w")
conv_study1d = open("conv_study1d.txt", "w")

H1_error_3d   = np.ones(6)
H1_rates_3d   = np.ones(6)
L2_error_3d   = np.ones(6)
L2_rates_3d   = np.ones(6)
err_interface     = np.ones(6)
rates_interface   = np.ones(6)
h_out             = np.ones(6)

H1_error_1d = np.ones(6)
H1_rates_1d = np.ones(6)
L2_error_1d = np.ones(6)
L2_rates_1d = np.ones(6)

kappa = 1.0
gamma_in = 1.0
gamma_out = 1.0

mesh_vec = [0,1,2,3,4]

for i in mesh_vec:
  ncell = 4*(2**i)
  mesh_3d = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),ncell,ncell,ncell)
  radius = 0.05

  # === Perimeter and Area ===
  D_perimeter = 2*np.pi*radius
  D_area = np.pi*(radius**2)

  # === Create Meshes ===
  Omega = MeshFunction("size_t",mesh_3d,1,0)
  CompiledSubDomain("near(x[0],0.0) && near(x[1],0.0)").mark(Omega,1)
  Lambda = EmbeddedMesh(Omega,1)

  # === Function Spaces ===
  V3 = FunctionSpace(mesh_3d, "CG", 1)
  V1 = FunctionSpace(Lambda, "DG", 1)
  W = [V3, V1]
  u3, u1 = map(TrialFunction, W)
  v3, v1 = map(TestFunction, W)

  # === Mesh Measures ===
  dxOmega = Measure("dx", domain=mesh_3d)
  dxLambda = Measure("dx", domain=Lambda)
  dsLambda = Measure("ds", domain=Lambda)
  #N = FacetNormal(mesh_3d)
  n = FacetNormal(Lambda)
  h = CellDiameter(Lambda)
  alpha = Constant(30.0)
  epsilon = Constant(-1.0)
  perf1 = 1.0
  perf3 = 1.0

  # === Cylinder ===
  cylinder = Circle(radius=radius, degree=20)

  # === Averages ===
  u3_avg = Average(u3, Lambda, cylinder)
  v3_avg = Average(v3, Lambda, cylinder)


  # === Exact Solutions ===
  r = Expression('sqrt(pow(x[0] - 0.0, 2)+ pow(x[1] - 0.0,2))', degree  = 7)
  u3_exact_expr = Expression('r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*(sin(pi*x[2])+2) : (kappa/(1+kappa))*(sin(pi*x[2])+2)',
                             degree = 7,
                             radius = radius,
                             kappa = kappa ,
                             r= r)
  u3_exact = interpolate(u3_exact_expr, V3)

  u1_exact_expr = Expression('(sin(pi*x[2])+2)', degree = 7)
  u1_exact = interpolate(u1_exact_expr, V1)

  vtkfile = File('u3_exact.pvd')
  vtkfile << u3_exact
  vtkfile = File('u1_exact.pvd')
  vtkfile << u1_exact

  # === Forcing terms ===
  f3 = Expression('r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pow(pi,2)*sin(pi*x[2]) :(kappa/(1+kappa))*pow(pi,2)*sin(pi*x[2])',
                  degree = 4,
                  kappa = kappa,
                  radius = radius,
                  r= r)

  f1 = Expression('area*sin(pi*x[2])*pow(pi,2) + (kappa/(1+kappa))*2*pi*radius*(sin(pi*x[2])+2)',
                  degree = 4,
                  radius = radius,
                  kappa = kappa,
                  area = D_area)

  # === Variational Forms ===
  a00 = inner(grad(u3), grad(v3)) * dxOmega + D_perimeter * kappa * inner(u3_avg, v3_avg) * dxLambda
  a01 = -kappa * D_perimeter * inner(u1, v3_avg) * dxLambda
  a10 = -kappa * D_perimeter * inner(u3_avg, v1) * dxLambda
  a11 = D_area * inner(grad(u1), grad(v1)) * dxLambda \
      - D_area * inner(avg(grad(u1)), jump(v1, n)) * dS \
      - D_area * inner(avg(grad(v1)), jump(u1, n)) * dS \
      + (alpha/avg(h)) * inner(jump(u1), jump(v1)) * dS \
      + kappa * D_perimeter * inner(u1, v1) * dxLambda \
      - gamma_in * inner(u1, v1) * D_area * dsLambda(1) \
      + gamma_out * inner(u1, v1) * D_area * dsLambda(2)

  L0 = inner(f3, v3) * dxOmega
  L1 = inner(f1, v1) * dxLambda \
      - gamma_in * inner(u1_exact, v1) * D_area * dsLambda(1) \
      + gamma_out * inner(u1_exact, v1) * D_area * dsLambda(2)

  a = [[a00, a01], [a10, a11]]
  L = [L0, L1]

  bc3d = DirichletBC(V3, u3_exact, "on_boundary")
  W_bcs = [bc3d, []]  # No Dirichlet BCs on 1D

  A, b = map(ii_assemble, (a, L))
  A, b = apply_bc(A, b, W_bcs)
  A, b = map(ii_convert, (A, b))

  solver = PETScKrylovSolver()
  solver.set_operators(A, A)
  ksp = solver.ksp()

  opts = PETSc.Options()
  opts.setValue('ksp_type', 'gmres')
  opts.setValue('ksp_norm_type', 'unpreconditioned')
  opts.setValue('ksp_atol', 1E-14)
  opts.setValue('ksp_rtol', 1E-30)
  opts.setValue('ksp_monitor_true_residual', None)
  opts.setValue('pc_type', 'hypre')
  ksp.setFromOptions()

  x = b.copy()
  solver.solve(A,x,b)

  wh = ii_Function(W)
  wh.vector()[:] = x
  uh3d, uh1d = wh

  avg_uh3d = Average(uh3d, Lambda, cylinder)
  avg_u3d_exact = Average(u3_exact, Lambda, cylinder)

  # === Store Model Predictions ===
  vtkfile = File('3d_Prediction.pvd')
  vtkfile << uh3d

  vtkfile = File('1d_Prediction.pvd')
  vtkfile << uh1d


  # === Computer Errors ===
  h_out[i] =uh3d.function_space().mesh().hmax()
  H1_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'H1')
  L2_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'L2')
  err_interface[i] = 1.0
  H1_rates_3d[i] = ln(H1_error_3d[i]/H1_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_3d[i] = ln(L2_error_3d[i]/L2_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  rates_interface[i] = ln(err_interface[i]/err_interface[i-1])/ln(h_out[i]/h_out[i-1])

  H1_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'H1')
  L2_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'L2')
  H1_rates_1d[i] = ln(H1_error_1d[i]/H1_error_1d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_1d[i] = ln(L2_error_1d[i]/L2_error_1d[i-1])/ln(h_out[i]/h_out[i-1])

  print(i,"{:.3e}".format(h_out[i]),"{:.3e}".format(H1_error_3d[i]), "{:.3e}".format(H1_rates_3d[i]), "{:.3e}".format(err_interface[i]),"{:.3e}".format(rates_interface[i]), "{:.3e}".format(L2_error_3d[i]),"{:.3e}".format(L2_rates_3d[i]),  file= conv_study3d,flush = True)
  print(i,"{:.3e}".format(ncell),"{:.3e}".format(H1_error_1d[i]), "{:.3e}".format(H1_rates_1d[i]), "{:.3e}".format(L2_error_1d[i]),"{:.3e}".format(L2_rates_1d[i]),  file= conv_study1d,flush = True)





Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_cdd21d347a8b939a40c5cd04a75bdb0e0e4b85cb

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00209618 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.207994 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:----------------------------

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_99e64e64a0b0bf7efa2745c3166b98a12c5ca04c

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00144577 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0277143 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:---------------------------

Calling FFC just-in-time (JIT) compiler, this may take some time.


...

  0 KSP unpreconditioned resid norm 3.766776560745e+01 true resid norm 3.766776560745e+01 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.090434817848e-02 true resid norm 2.090434817841e-02 ||r(i)||/||b|| 5.549665036218e-04
  2 KSP unpreconditioned resid norm 5.548349946365e-04 true resid norm 5.548349944935e-04 ||r(i)||/||b|| 1.472970285192e-05
  3 KSP unpreconditioned resid norm 1.014837242894e-05 true resid norm 1.014837245612e-05 ||r(i)||/||b|| 2.694179570374e-07
  4 KSP unpreconditioned resid norm 1.196745387170e-07 true resid norm 1.196745181400e-07 ||r(i)||/||b|| 3.177106903211e-09
  5 KSP unpreconditioned resid norm 1.638138921317e-09 true resid norm 1.638174896349e-09 ||r(i)||/||b|| 4.349010008773e-11
  6 KSP unpreconditioned resid norm 1.970429973004e-11 true resid norm 1.973897443211e-11 ||r(i)||/||b|| 5.240282802495e-13
  7 KSP unpreconditioned resid norm 1.835958378389e-13 true resid norm 9.085678283876e-13 ||r(i)||/||b|| 2.412056605258e-14
  8 KSP 

Averaging over 32 cells: 100%|██████████| 32/32 [00:00<00:00, 475.52it/s]


  0 KSP unpreconditioned resid norm 7.517610382336e+01 true resid norm 7.517610382336e+01 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.099341377936e-02 true resid norm 2.099341377927e-02 ||r(i)||/||b|| 2.792564752836e-04
  2 KSP unpreconditioned resid norm 5.117431894116e-04 true resid norm 5.117431892807e-04 ||r(i)||/||b|| 6.807258733216e-06
  3 KSP unpreconditioned resid norm 1.266395878480e-05 true resid norm 1.266395878625e-05 ||r(i)||/||b|| 1.684572376350e-07
  4 KSP unpreconditioned resid norm 2.021205448863e-07 true resid norm 2.021206829501e-07 ||r(i)||/||b|| 2.688629400441e-09
  5 KSP unpreconditioned resid norm 2.824098803178e-09 true resid norm 2.824175252224e-09 ||r(i)||/||b|| 3.756745971911e-11
  6 KSP unpreconditioned resid norm 3.828448002578e-11 true resid norm 3.833499008060e-11 ||r(i)||/||b|| 5.099358457134e-13
  7 KSP unpreconditioned resid norm 5.254732671780e-13 true resid norm 2.627578249164e-12 ||r(i)||/||b|| 3.495230685721e-14
  8 KSP 

Averaging over 64 cells: 100%|██████████| 64/64 [00:00<00:00, 477.49it/s]


  0 KSP unpreconditioned resid norm 1.502766998004e+02 true resid norm 1.502766998004e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.940225691703e-02 true resid norm 1.940225691703e-02 ||r(i)||/||b|| 1.291102143100e-04
  2 KSP unpreconditioned resid norm 4.180141608181e-04 true resid norm 4.180141609007e-04 ||r(i)||/||b|| 2.781629896424e-06
  3 KSP unpreconditioned resid norm 7.539587494764e-06 true resid norm 7.539587621723e-06 ||r(i)||/||b|| 5.017136809458e-08
  4 KSP unpreconditioned resid norm 1.410628446443e-07 true resid norm 1.410628582269e-07 ||r(i)||/||b|| 9.386874905709e-10
  5 KSP unpreconditioned resid norm 2.222482044969e-09 true resid norm 2.222568090447e-09 ||r(i)||/||b|| 1.478983830094e-11
  6 KSP unpreconditioned resid norm 3.518988785799e-11 true resid norm 3.618067231536e-11 ||r(i)||/||b|| 2.407603598124e-13
  7 KSP unpreconditioned resid norm 5.494057835570e-13 true resid norm 7.453113317220e-12 ||r(i)||/||b|| 4.959593421417e-14
  8 KSP 

In [ ]:
from math import gamma
from argparse import RawDescriptionHelpFormatter
from petsc4py import PETSc
from ast import Constant
from dolfin import *
from xii import *
import numpy as np
import time

# @title Convergence test: time dependent 3D-FEM, 1D-dG

conv_study3d = open("conv_study3d.txt", "w")
conv_study1d = open("conv_study1d.txt", "w")

H1_error_3d   = np.ones(6)
H1_rates_3d   = np.ones(6)
L2_error_3d   = np.ones(6)
L2_rates_3d   = np.ones(6)
err_interface     = np.ones(6)
rates_interface   = np.ones(6)
h_out             = np.ones(6)

H1_error_1d = np.ones(6)
H1_rates_1d = np.ones(6)
L2_error_1d = np.ones(6)
L2_rates_1d = np.ones(6)

kappa     = 1.0
gamma_in  = 1.0
gamma_out = 1.0

mesh_vec = [0,1,2,3,4]

T    = 1.0
#dt   = 0.1
#tpts = int(T/dt)
#inv_dt = Constant(1.0/dt)

for i in mesh_vec:
  ncell   = 4*(2**i)
  mesh_3d = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),ncell,ncell,ncell)
  radius  = 0.05
  dt      = 0.1#/ncell
  inv_dt  = Constant(1.0/dt)
  tpts    = int(T/dt)

  # === Perimeter and Area ===
  D_perimeter = 2*np.pi*radius
  D_area = np.pi*(radius**2)

  # === Create Meshes ===
  Omega = MeshFunction("size_t",mesh_3d,1,0)
  CompiledSubDomain("near(x[0],0.0) && near(x[1],0.0)").mark(Omega,1)
  Lambda = EmbeddedMesh(Omega,1)

  # === Function Spaces ===
  V3 = FunctionSpace(mesh_3d, "CG", 1)
  V1 = FunctionSpace(Lambda, "DG", 1)
  W = [V3, V1]
  u3, u1 = map(TrialFunction, W)
  v3, v1 = map(TestFunction, W)

  # === Mesh Measures ===
  dxOmega = Measure("dx", domain=mesh_3d)
  dxLambda = Measure("dx", domain=Lambda)
  dsLambda = Measure("ds", domain=Lambda)
  #N = FacetNormal(mesh_3d)
  n = FacetNormal(Lambda)
  h = CellDiameter(Lambda)
  alpha = Constant(30.0)
  epsilon = Constant(-1.0)
  perf1 = 1.0
  perf3 = 1.0

  # === Cylinder ===
  cylinder = Circle(radius=radius, degree=20)

  # === Averages ===
  u3_avg = Average(u3, Lambda, cylinder)
  v3_avg = Average(v3, Lambda, cylinder)

  # === Initialize time step ===
  uh3d_prev = interpolate(Expression('0.0',degree=1), V3)
  uh1d_prev = interpolate(Expression('0.0',degree=1), V1)

  r = Expression('sqrt(pow(x[0] - 0.0, 2)+ pow(x[1] - 0.0,2))', degree  = 7)

  # === Begin time-stepping ===
  t = 0.0
  for step in range(tpts):
    t += dt
    print(f"[INFO] Time step {step+1}/{tpts}, t = {t:.3}")

    # === Exact solutions ===

    u3_exact_expr = Expression('(r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*((sin(pi*x[2])+2)*t) : (kappa/(1+kappa))*((sin(pi*x[2])+2)*t))',
                              degree = 7,
                              radius = radius,
                              kappa = kappa ,
                              r = r,
                              t = t)
    u3_exact = interpolate(u3_exact_expr, V3)

    u1_exact_expr = Expression('(sin(pi*x[2])+2)*t', degree = 7, t=t)
    u1_exact = interpolate(u1_exact_expr, V1)

    vtkfile = File('u3_exact.pvd')
    vtkfile << u3_exact
    vtkfile = File('u1_exact.pvd')
    vtkfile << u1_exact

    # === Forcing terms ===
    f3 = Expression('(r > radius ? area*(kappa/(1+kappa))*(1-radius*std::log(r/radius))*(sin(pi*x[2])+2) + (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pow(pi,2)*sin(pi*x[2])*t :(kappa/(1+kappa))*pow(pi,2)*sin(pi*x[2]))*t + area*(kappa/(1+kappa))*(sin(pi*x[2])+2)',
                    degree = 4,
                    kappa = kappa,
                    radius = radius,
                    area = D_area,
                    r = r,
                    t = t)

    f1 = Expression(
        'area * (sin(pi*x[2])+2)'                             # time derivative
        ' + area * pow(pi,2) * t * sin(pi*x[2])'               # diffusion
        ' + (kappa / (1 + kappa)) * perimeter * t * (sin(pi*x[2]) + 2)',  # coupling
        degree=4,
        area=D_area,
        perimeter=D_perimeter,
        kappa=kappa,
        t=t
    )

    # === Checks for force terms ===
    #f3_func = interpolate(f3, V3)
    #f1_func = interpolate(f1, V1)
    #File(f"f3_step{step:03d}.pvd") << f3_func
    #File(f"f1_step{step:03d}.pvd") << f1_func

    # === Variational Forms ===
    a00 = D_area * inv_dt * (inner(u3, v3) * dxOmega) + inner(grad(u3), grad(v3)) * dxOmega + D_perimeter * kappa * inner(u3_avg, v3_avg) * dxLambda
    a01 = -kappa * D_perimeter * inner(u1, v3_avg) * dxLambda
    a10 = -kappa * D_perimeter * inner(u3_avg, v1) * dxLambda
    a11 = D_area * inv_dt * inner(u1, v1) * dxLambda + D_area * inner(grad(u1), grad(v1)) * dxLambda \
        - D_area * inner(avg(grad(u1)), jump(v1, n)) * dS \
        - D_area * inner(avg(grad(v1)), jump(u1, n)) * dS \
        + (alpha/avg(h)) * inner(jump(u1), jump(v1)) * dS \
        + kappa * D_perimeter * inner(u1, v1) * dxLambda
        #- gamma_in * inner(u1, v1) * D_area * dsLambda(1)
        #+ gamma_out * inner(u1, v1) * D_area * dsLambda(2)

    L0 = inner(f3, v3) * dxOmega + D_area * inv_dt * (inner(uh3d_prev, v3) * dxOmega)
    L1 = inner(f1, v1) * dxLambda + D_area * inv_dt * (inner(uh1d_prev, v1) * dxLambda)
        #- gamma_in * inner(u3_exact_expr, v1) * area * dsLambda(1) \
        #+ gamma_out * inner(u3_exact_expr, v1) * area * dsLambda(2)

    a = [[a00, a01], [a10, a11]]
    L = [L0, L1]

    bc3d = DirichletBC(V3, u3_exact, "on_boundary")
    W_bcs = [bc3d, []]  # No Dirichlet BCs on 1D

    A, b = map(ii_assemble, (a, L))
    A, b = apply_bc(A, b, W_bcs)
    A, b = map(ii_convert, (A, b))

    solver = PETScKrylovSolver()
    solver.set_operators(A, A)
    ksp = solver.ksp()

    opts = PETSc.Options()
    opts.setValue('ksp_type', 'gmres')
    opts.setValue('ksp_norm_type', 'unpreconditioned')
    opts.setValue('ksp_atol', 1E-14)
    opts.setValue('ksp_rtol', 1E-30)
    opts.setValue('ksp_monitor_true_residual', None)
    opts.setValue('pc_type', 'hypre')
    ksp.setFromOptions()

    x = b.copy()
    solver.solve(A,x,b)

    wh = ii_Function(W)
    wh.vector()[:] = x
    uh3d, uh1d = wh

    uh3d_prev.assign(uh3d)
    uh1d_prev.assign(uh1d)

    avg_uh3d = Average(uh3d, Lambda, cylinder)
    avg_u3d_exact = Average(u3_exact, Lambda, cylinder)

    # === Store Model Predictions ===
    vtkfile = File('3d_Prediction.pvd')
    vtkfile << uh3d

    vtkfile = File('1d_Prediction.pvd')
    vtkfile << uh1d


  # === Computer Errors ===
  h_out[i] =uh3d.function_space().mesh().hmax()
  H1_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'H1')
  L2_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'L2')
  err_interface[i] = 1.0
  H1_rates_3d[i] = ln(H1_error_3d[i]/H1_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_3d[i] = ln(L2_error_3d[i]/L2_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  rates_interface[i] = ln(err_interface[i]/err_interface[i-1])/ln(h_out[i]/h_out[i-1])

  H1_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'H1')
  L2_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'L2')
  H1_rates_1d[i] = ln(H1_error_1d[i]/H1_error_1d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_1d[i] = ln(L2_error_1d[i]/L2_error_1d[i-1])/ln(h_out[i]/h_out[i-1])

  print(i,"{:.3e}".format(h_out[i]),"{:.3e}".format(H1_error_3d[i]), "{:.3e}".format(H1_rates_3d[i]), "{:.3e}".format(err_interface[i]),"{:.3e}".format(rates_interface[i]), "{:.3e}".format(L2_error_3d[i]),"{:.3e}".format(L2_rates_3d[i]),  file= conv_study3d,flush = True)
  print(i,"{:.3e}".format(ncell),"{:.3e}".format(H1_error_1d[i]), "{:.3e}".format(H1_rates_1d[i]), "{:.3e}".format(L2_error_1d[i]),"{:.3e}".format(L2_rates_1d[i]),  file= conv_study1d,flush = True)

  # ===== Print Convergence Table =====
  print("\nTable 5.1-like Results (Zunino Setup)")
  print(f"{'N':>4} | {'||e3||_H1':>10} | {'Rate':>6} | {'||e3||_L2':>10} | {'Rate':>6} | {'||e1||_H1':>10} | {'Rate':>6} | {'||e1||_L2':>10} | {'Rate':>6}")
  print("-"*90)



[INFO] Time step 1/10, t = 0.1


Averaging over 4 cells: 100%|██████████| 4/4 [00:00<00:00, 408.26it/s]

  0 KSP unpreconditioned resid norm 9.860966824855e-01 true resid norm 9.860966824855e-01 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 3.831291342531e-04 true resid norm 3.831291342525e-04 ||r(i)||/||b|| 3.885310041677e-04
  2 KSP unpreconditioned resid norm 2.337788164906e-06 true resid norm 2.337788161561e-06 ||r(i)||/||b|| 2.370749443826e-06
  3 KSP unpreconditioned resid norm 4.778194432893e-10 true resid norm 4.778235463883e-10 ||r(i)||/||b|| 4.845605455075e-10
  4 KSP unpreconditioned resid norm 1.028954635550e-14 true resid norm 1.061173339142e-14 ||r(i)||/||b|| 1.076135188354e-14
  5 KSP unpreconditioned resid norm 1.406898523278e-16 true resid norm 8.000672591671e-15 ||r(i)||/||b|| 8.113476836273e-15
[INFO] Time step 2/10, t = 0.2
  0 KSP unpreconditioned resid norm 1.972563341890e+00 true resid norm 1.972563341890e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 7.659109938359e-04 true resid norm 7.659109938327e-04 ||r(i)||/||

  0 KSP unpreconditioned resid norm 6.911168661144e+00 true resid norm 6.911168661144e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.686484114769e-03 true resid norm 2.686484114806e-03 ||r(i)||/||b|| 3.887163295421e-04
  2 KSP unpreconditioned resid norm 1.638939616173e-05 true resid norm 1.638939615937e-05 ||r(i)||/||b|| 2.371436288556e-06
  3 KSP unpreconditioned resid norm 3.360556399173e-09 true resid norm 3.360520343236e-09 ||r(i)||/||b|| 4.862448752162e-10
  4 KSP unpreconditioned resid norm 7.593371556122e-14 true resid norm 7.478963212295e-14 ||r(i)||/||b|| 1.082156083723e-14
  5 KSP unpreconditioned resid norm 6.449673590950e-16 true resid norm 4.273573184384e-14 ||r(i)||/||b|| 6.183575302410e-15
[INFO] Time step 8/10, t = 0.8
  0 KSP unpreconditioned resid norm 7.900229935065e+00 true resid norm 7.900229935065e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 3.072111509506e-03 true resid norm 3.072111509490e-03 ||r(i)||/||

Averaging over 8 cells: 100%|██████████| 8/8 [00:00<00:00, 314.82it/s]


...

  0 KSP unpreconditioned resid norm 3.766723097900e+00 true resid norm 3.766723097900e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.953493134202e-03 true resid norm 1.953493134204e-03 ||r(i)||/||b|| 5.186187259936e-04
  2 KSP unpreconditioned resid norm 2.671514392488e-05 true resid norm 2.671514391934e-05 ||r(i)||/||b|| 7.092409828115e-06
  3 KSP unpreconditioned resid norm 3.549043073143e-07 true resid norm 3.549042955130e-07 ||r(i)||/||b|| 9.422096774536e-08
  4 KSP unpreconditioned resid norm 4.025118598110e-09 true resid norm 4.025121719867e-09 ||r(i)||/||b|| 1.068600376309e-09
  5 KSP unpreconditioned resid norm 5.340676861040e-11 true resid norm 5.341201079530e-11 ||r(i)||/||b|| 1.417996741652e-11
  6 KSP unpreconditioned resid norm 6.027511821327e-13 true resid norm 6.081548566995e-13 ||r(i)||/||b|| 1.614546227299e-13
  7 KSP unpreconditioned resid norm 7.538609395537e-15 true resid norm 8.098033408665e-14 ||r(i)||/||b|| 2.149888164909e-14
[INFO] T

Averaging over 32 cells: 100%|██████████| 32/32 [00:00<00:00, 549.78it/s]


  0 KSP unpreconditioned resid norm 7.517603882641e+00 true resid norm 7.517603882641e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.039197695340e-03 true resid norm 2.039197695361e-03 ||r(i)||/||b|| 2.712563374175e-04
  2 KSP unpreconditioned resid norm 3.406853934908e-05 true resid norm 3.406853936361e-05 ||r(i)||/||b|| 4.531834863270e-06
  3 KSP unpreconditioned resid norm 4.278950663836e-07 true resid norm 4.278950610494e-07 ||r(i)||/||b|| 5.691907524383e-08
  4 KSP unpreconditioned resid norm 5.513439223035e-09 true resid norm 5.513428056630e-09 ||r(i)||/||b|| 7.334023104570e-10
  5 KSP unpreconditioned resid norm 8.151809763562e-11 true resid norm 8.148473708752e-11 ||r(i)||/||b|| 1.083919003443e-11
  6 KSP unpreconditioned resid norm 1.210009729304e-12 true resid norm 1.212637135731e-12 ||r(i)||/||b|| 1.613063357236e-13
  7 KSP unpreconditioned resid norm 1.657877275649e-14 true resid norm 3.085447780649e-13 ||r(i)||/||b|| 4.104296832896e-14
  8 KSP 

Averaging over 64 cells: 100%|██████████| 64/64 [00:00<00:00, 532.02it/s]


  0 KSP unpreconditioned resid norm 1.502766932479e+01 true resid norm 1.502766932479e+01 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.927972202448e-03 true resid norm 1.927972202452e-03 ||r(i)||/||b|| 1.282948247518e-04
  2 KSP unpreconditioned resid norm 3.903914723277e-05 true resid norm 3.903914723643e-05 ||r(i)||/||b|| 2.597817824753e-06
  3 KSP unpreconditioned resid norm 6.926936816339e-07 true resid norm 6.926936713374e-07 ||r(i)||/||b|| 4.609455108217e-08
  4 KSP unpreconditioned resid norm 1.338260207483e-08 true resid norm 1.338259277899e-08 ||r(i)||/||b|| 8.905301607160e-10
  5 KSP unpreconditioned resid norm 2.014192955373e-10 true resid norm 2.014312214018e-10 ||r(i)||/||b|| 1.340402274287e-11
  6 KSP unpreconditioned resid norm 3.147524858898e-12 true resid norm 3.234857449586e-12 ||r(i)||/||b|| 2.152600898830e-13
  7 KSP unpreconditioned resid norm 4.995804743488e-14 true resid norm 7.195332669649e-13 ||r(i)||/||b|| 4.788056294119e-14
  8 KSP 

In [ ]:
from math import gamma
from argparse import RawDescriptionHelpFormatter
from petsc4py import PETSc
from ast import Constant
from dolfin import *
from xii import *
import numpy as np
import time

# @title Convergence test: 3D-FEM,1D-dG time-dependent advection-diffusion

conv_study3d = open("conv_study3d.txt", "w")
conv_study1d = open("conv_study1d.txt", "w")

H1_error_3d   = np.ones(6)
H1_rates_3d   = np.ones(6)
L2_error_3d   = np.ones(6)
L2_rates_3d   = np.ones(6)
err_interface     = np.ones(6)
rates_interface   = np.ones(6)
h_out             = np.ones(6)

H1_error_1d = np.ones(6)
H1_rates_1d = np.ones(6)
L2_error_1d = np.ones(6)
L2_rates_1d = np.ones(6)

kappa     = 1.0
gamma_in  = 1.0
gamma_out = 1.0

mesh_vec = [0,1,2,3,4]

T    = 1.0
#dt   = 0.1
#tpts = int(T/dt)
#inv_dt = Constant(1.0/dt)

for i in mesh_vec:
  ncell   = 4*(2**i)
  mesh_3d = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),ncell,ncell,ncell)
  radius  = 0.05
  dt      = 0.1#/ncell
  inv_dt  = Constant(1.0/dt)
  tpts    = int(T/dt)

  # === Perimeter and Area ===
  D_perimeter = 2*np.pi*radius
  D_area = np.pi*(radius**2)

  # === Create Meshes ===
  Omega = MeshFunction("size_t",mesh_3d,1,0)
  CompiledSubDomain("near(x[0],0.0) && near(x[1],0.0)").mark(Omega,1)
  Lambda = EmbeddedMesh(Omega,1)
  Lambda_boundaries = MeshFunction("size_t", Lambda, Lambda.topology().dim() - 1, 0)
  CompiledSubDomain("near(x[2], -0.5)").mark(Lambda_boundaries, 1)
  CompiledSubDomain("near(x[2],  0.5)").mark(Lambda_boundaries, 2)

  # === Function Spaces ===
  V3 = FunctionSpace(mesh_3d, "CG", 1)
  V1 = FunctionSpace(Lambda, "DG", 1)
  W = [V3, V1]
  u3, u1 = map(TrialFunction, W)
  v3, v1 = map(TestFunction, W)

  # === Velocity terms ===
  q1 = Constant(1.0)
  q3 = as_vector((0,0,q1))

  # === Initial concentration ===
  cinlet = Expression("(sin(pi*x[2])+2)*t", degree=4, t=0.0)#Constant(1.0)


  # === Mesh Measures ===
  dxOmega = Measure("dx", domain=mesh_3d)
  dxLambda = Measure("dx", domain=Lambda)
  dsLambda = Measure("ds", domain=Lambda, subdomain_data=Lambda_boundaries)
  #dsLambda = Measure("ds", domain=Lambda)
  #N = FacetNormal(mesh_3d)
  n = FacetNormal(Lambda)
  h = CellDiameter(Lambda)
  alpha = Constant(50.0)
  epsilon = Constant(-1.0)
  perf1 = 1.0
  perf3 = 1.0

  # === Upwind ===
  #u1_up = u1('+')
  #v1_up = v1('-')
  #q3_con = Constant((0,0,q1))
  #q1_up = (dot(q3_con, n) + abs(dot(q3_con, n))) / 2.0
  q1_up = (q1 + abs(q1))/2.0 # works best, gets okay convergence - rates are off

  # === Cylinder ===
  cylinder = Circle(radius=radius, degree=20)

  # === Averages ===
  u3_avg = Average(u3, Lambda, cylinder)
  v3_avg = Average(v3, Lambda, cylinder)

  # === Initialize time step ===
  uh3d_prev = interpolate(Expression('0.0',degree=1), V3)
  uh1d_prev = interpolate(Expression('0.0',degree=1), V1)

  r = Expression('sqrt(pow(x[0] - 0.0, 2)+ pow(x[1] - 0.0,2))', degree  = 7)

  # === Begin time-stepping ===
  t = 0.0
  for step in range(tpts):
    t += dt
    print(f"[INFO] Time step {step+1}/{tpts}, t = {t:.3}")

    cinlet.t = t
    # === Exact solutions ===

    u3_exact_expr = Expression('(r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*((sin(pi*x[2])+2)*t) : (kappa/(1+kappa))*((sin(pi*x[2])+2)*t))',
                              degree = 7,
                              radius = radius,
                              kappa = kappa ,
                              r = r,
                              t = t)
    u3_exact = interpolate(u3_exact_expr, V3)

    u1_exact_expr = Expression('(sin(pi*x[2])+2)*t', degree = 7, t=t)
    u1_exact = interpolate(u1_exact_expr, V1)

    vtkfile = File('u3_exact.pvd')
    vtkfile << u3_exact
    vtkfile = File('u1_exact.pvd')
    vtkfile << u1_exact

    # === Forcing terms ===
    f3 = Expression('(r > radius ? area*(kappa/(1+kappa))*(1-radius*std::log(r/radius))*(sin(pi*x[2])+2) + (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pow(pi,2)*sin(pi*x[2])*t + (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pi*t*q*cos(pi*x[2]) :(kappa/(1+kappa))*pow(pi,2)*sin(pi*x[2]))*t + area*(kappa/(1+kappa))*(sin(pi*x[2])+2 + (kappa/(1+kappa))*pi*t*q*cos(pi*x[2]))',
                    degree = 4,
                    kappa = kappa,
                    radius = radius,
                    area = D_area,
                    r = r,
                    t = t,
                    q = q1)

    f1 = Expression(
        'area * (sin(pi*x[2])+2)'                             # time derivative
        ' + area * pow(pi,2) * t * sin(pi*x[2])'               # diffusion
        ' + (kappa / (1 + kappa)) * perimeter * t * (sin(pi*x[2]) + 2)'  # coupling
        ' + area * pi * t * q * cos(pi*x[2])',                # advection
        degree=4,
        area=D_area,
        perimeter=D_perimeter,
        kappa=kappa,
        t=t,
        q=q1
    )

    # === Checks for force terms ===
    #f3_func = interpolate(f3, V3)
    #f1_func = interpolate(f1, V1)
    #File(f"f3_step{step:03d}.pvd") << f3_func
    #File(f"f1_step{step:03d}.pvd") << f1_func

    # === Variational Forms ===
    a00 = D_area * inv_dt * (inner(u3, v3) * dxOmega) + inner(grad(u3), grad(v3)) * dxOmega \
          + D_perimeter * kappa * inner(u3_avg, v3_avg) * dxLambda + inner(dot(q3,grad(u3)),v3) *dxOmega
    a01 = -kappa * D_perimeter * inner(u1, v3_avg) * dxLambda
    a10 = -kappa * D_perimeter * inner(u3_avg, v1) * dxLambda
    a11 = D_area * inv_dt * u1 * v1 * dxLambda \
        + D_area * inner(grad(u1), grad(v1)) * dxLambda \
        - D_area * inner(avg(grad(u1)), jump(v1, n)) * dS \
        - D_area * inner(avg(grad(v1)), jump(u1, n)) * dS \
        + (alpha/avg(h)) * jump(u1) * jump(v1) * dS \
        + kappa * D_perimeter * u1 * v1 * dxLambda \
        - D_area * q1 * u1 * v1.dx(2) * dxLambda \
        + D_area * jump(v1) * (q1_up * u1('+') - q1_up * u1('-')) * dS + D_area * q1_up * u1 * v1 * dsLambda(2) #* q1 * u1_up * jump(v1) * dS



    L0 = inner(f3, v3) * dxOmega + D_area * inv_dt * (inner(uh3d_prev, v3) * dxOmega)
    L1 = inner(f1, v1) * dxLambda \
        + D_area * inv_dt * uh1d_prev * v1 * dxLambda \
        + D_area * q1_up * cinlet * v1 * dsLambda(1)


    a = [[a00, a01], [a10, a11]]
    L = [L0, L1]

    bc3d = DirichletBC(V3, u3_exact, "on_boundary")
    W_bcs = [bc3d, []]  # No Dirichlet BCs on 1D

    A, b = map(ii_assemble, (a, L))
    A, b = apply_bc(A, b, W_bcs)
    A, b = map(ii_convert, (A, b))

    solver = PETScKrylovSolver()
    solver.set_operators(A, A)
    ksp = solver.ksp()

    opts = PETSc.Options()
    opts.setValue('ksp_type', 'gmres')
    opts.setValue('ksp_norm_type', 'unpreconditioned')
    opts.setValue('ksp_atol', 1E-14)
    opts.setValue('ksp_rtol', 1E-30)
    opts.setValue('ksp_monitor_true_residual', None)
    opts.setValue('pc_type', 'hypre')
    ksp.setFromOptions()

    x = b.copy()
    solver.solve(A,x,b)

    wh = ii_Function(W)
    wh.vector()[:] = x
    uh3d, uh1d = wh

    uh3d_prev.assign(uh3d)
    uh1d_prev.assign(uh1d)

    avg_uh3d = Average(uh3d, Lambda, cylinder)
    avg_u3d_exact = Average(u3_exact, Lambda, cylinder)

    # === Store Model Predictions ===
    vtkfile = File('3d_Prediction.pvd')
    vtkfile << uh3d

    vtkfile = File('1d_Prediction.pvd')
    vtkfile << uh1d


  # === Computer Errors ===
  h_out[i] =uh3d.function_space().mesh().hmax()
  H1_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'H1')
  L2_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'L2')
  err_interface[i] = 1.0
  H1_rates_3d[i] = ln(H1_error_3d[i]/H1_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_3d[i] = ln(L2_error_3d[i]/L2_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  rates_interface[i] = ln(err_interface[i]/err_interface[i-1])/ln(h_out[i]/h_out[i-1])

  H1_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'H1')
  L2_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'L2')
  H1_rates_1d[i] = ln(H1_error_1d[i]/H1_error_1d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_1d[i] = ln(L2_error_1d[i]/L2_error_1d[i-1])/ln(h_out[i]/h_out[i-1])

  print(i,"{:.3e}".format(h_out[i]),"{:.3e}".format(H1_error_3d[i]), "{:.3e}".format(H1_rates_3d[i]), "{:.3e}".format(err_interface[i]),"{:.3e}".format(rates_interface[i]), "{:.3e}".format(L2_error_3d[i]),"{:.3e}".format(L2_rates_3d[i]),  file= conv_study3d,flush = True)
  print(i,"{:.3e}".format(ncell),"{:.3e}".format(H1_error_1d[i]), "{:.3e}".format(H1_rates_1d[i]), "{:.3e}".format(L2_error_1d[i]),"{:.3e}".format(L2_rates_1d[i]),  file= conv_study1d,flush = True)




/usr/local/lib/python3.11/dist-packages/block/__init__.py:15: UserWarning: The cbc.block repository has moved to https://github.com/blocknics/cbc.block
  warnings.warn('The cbc.block repository has moved to https://github.com/blocknics/cbc.block', UserWarning)


Missing HsMG for fract norm computing
Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_cdd21d347a8b939a40c5cd04a75bdb0e0e4b85cb

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00152898 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.188397 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:----------------------------

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_99e64e64a0b0bf7efa2745c3166b98a12c5ca04c

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00175428 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0239761 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:---------------------------

...

Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_7566fafeb9a731c5389e5e86e5a5f3f3611f1ab6

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       3
  Number of cell subdomains: 0
  Rank:                      0
  Arguments:                 '()'
  Number of coefficients:    1
  Coefficients:              '[f_985]'
  Unique elements:           'DG4(?,?), Vector<3 x CG1(?,?)>'
  Unique sub elements:       'DG4(?,?), Vector<3 x CG1(?,?)>, CG1(?,?)'
  
INFO:FFC:  representation:    auto --> uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: auto --> 8
INFO:FFC:  quadrature_degree: 8
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.0133328 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:----------------------

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_8457036ca23329ffa538d47158d469398d7664fe

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00142479 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.00623441 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representat

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-770867548.py", line 228, in <cell line: 0>
    H1_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'H1')
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/dolfin/fem/norms.py", line 280, in errornorm
    return norm(e, norm_type=norm_type, mesh=mesh)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/dolfin/fem/norms.py", line 154, in norm
    return sqrt(assemble_func(M))
                ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/dolfin/fem/assembling.py", line 202, in assemble
    dolfin_form = _create_dolfin_form(form, form_compiler_parameters)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.1

TypeError: object of type 'NoneType' has no len()

In [ ]:
# @title sanity checks

values = uh1d.vector().get_local()
print(f"1D pressure range: min = {np.min(values):.3e}, max = {np.max(values):.3e}")
values = u1_exact.vector().get_local()
print(f"Exact 1D pressure range: min = {np.min(values):.3e}, max = {np.max(values):.3e}")
values = uh3d.vector().get_local()
print(f"3D pressure range: min = {np.min(values):.3e}, max = {np.max(values):.3e}")
values = u3_exact.vector().get_local()
print(f"Exact 3D pressure range: min = {np.min(values):.3e}, max = {np.max(values):.3e}")

print("Min/max 3D pressure:", uh3d.vector().min(), uh3d.vector().max())
print("Min/max 1D pressure:", uh1d.vector().min(), uh1d.vector().max())


1D pressure range: min = 9.981e-01, max = 2.994e+00
Exact 1D pressure range: min = 1.000e+00, max = 3.000e+00
3D pressure range: min = 4.338e-01, max = 1.500e+00
Exact 3D pressure range: min = 4.338e-01, max = 1.500e+00
Min/max 3D pressure: 0.43377103291814956 1.5003724255703088
Min/max 1D pressure: 0.9981317108402998 2.9941614628324333


In [ ]:
# @title export

# Save numerical 3D solution
uh3d.rename("uh3", "numerical 3D")
File("output/uh3.pvd") << uh3d

# Save exact 3D solution
u3_exact_func = interpolate(u3_exact, uh3d.function_space())
u3_exact_func.rename("u3_exact", "exact 3D")
File("output/u3_exact.pvd") << u3_exact_func

# Save numerical 1D solution
uh1d.rename("uh1", "numerical 1D")
File("output/uh1.pvd") << uh1d

# Save exact 1D solution
u1_exact_func = interpolate(u1_exact, uh1d.function_space())
u1_exact_func.rename("u1_exact", "exact 1D")
File("output/u1_exact.pvd") << u1_exact_func


for z in z_vals:
    point = np.array([x_center, y_center, z])
    u1_exact_values.append(u1_exact_expr(point))
    uh1d_values.append(uh1d(point))

# Convert to numpy arrays
u1_exact_values = np.array(u1_exact_values)
uh1d_values = np.array(uh1d_values)

# Plot
plt.figure(figsize=(8,6))
plt.plot(z_vals, u1_exact_values, label='Exact Solution', linestyle='--')
plt.plot(z_vals, uh1d_values, label='Numerical Solution', marker='o', markersize=3, linestyle='-')
plt.xlabel('z')
plt.ylabel('Pressure at (x=0.5, y=0.5)')
plt.title('Centerline Pressure Comparison (Exact vs Numerical 1D)')
plt.legend()
plt.grid(True)
plt.show()

# Sample points along z-axis at (x=0.5, y=0.5)
num_samples = 100
z_vals = np.linspace(0, 1, num_samples)
x_center = 0.5
y_center = 0.5

u3_exact_values = []
uh3d_values = []

for z in z_vals:
    point = np.array([x_center, y_center, z])
    u3_exact_values.append(u3_exact(point))
    uh3d_values.append(uh3d(point))

# Convert to numpy arrays
u3_exact_values = np.array(u3_exact_values)
uh3d_values = np.array(uh3d_values)

# Plot
plt.figure(figsize=(8,6))
plt.plot(z_vals, u3_exact_values, label='Exact Solution', linestyle='--')
plt.plot(z_vals, uh3d_values, label='Numerical Solution', marker='o', markersize=3, linestyle='-')
plt.xlabel('z')
plt.ylabel('Pressure at (x=0.5, y=0.5)')
plt.title('Centerline Pressure Comparison (Exact vs Numerical 3D)')
plt.legend()
plt.grid(True)
plt.show()


# Sample points radially outward from center at z=0.5
num_samples_r = 100
r_vals = np.linspace(0, 0.5, num_samples_r)
z_fixed = 0.5

u3_exact_radial = []
uh3d_radial = []

for r in r_vals:
    x = 0.5 + r
    y = 0.5
    point = np.array([x, y, z_fixed])
    u3_exact_radial.append(u3_exact(point))
    uh3d_radial.append(uh3d(point))

# Convert to numpy arrays
u3_exact_radial = np.array(u3_exact_radial)
uh3d_radial = np.array(uh3d_radial)

# Plot
plt.figure(figsize=(8,6))
plt.plot(r_vals, u3_exact_radial, label='Exact Solution', linestyle='--')
plt.plot(r_vals, uh3d_radial, label='Numerical Solution', marker='o', markersize=3, linestyle='-')
plt.xlabel('Radial distance r from center (z=0.5)')
plt.ylabel('Pressure')
plt.title('Radial Pressure Comparison (Exact vs Numerical at z=0.5)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from math import gamma
from argparse import RawDescriptionHelpFormatter
from petsc4py import PETSc
from ast import Constant
from dolfin import *
from xii import *
import numpy as np
import time

# @title Testing

conv_study3d = open("conv_study3d.txt", "w")
conv_study1d = open("conv_study1d.txt", "w")

H1_error_3d   = np.ones(6)
H1_rates_3d   = np.ones(6)
L2_error_3d   = np.ones(6)
L2_rates_3d   = np.ones(6)
err_interface     = np.ones(6)
rates_interface   = np.ones(6)
h_out             = np.ones(6)

H1_error_1d = np.ones(6)
H1_rates_1d = np.ones(6)
L2_error_1d = np.ones(6)
L2_rates_1d = np.ones(6)

kappa     = 1.0
gamma_in  = 1.0
gamma_out = 1.0

mesh_vec = [0,1,2,3]

T    = 1.0
#dt   = 0.1
#tpts = int(T/dt)
#inv_dt = Constant(1.0/dt)

for i in mesh_vec:
  ncell   = 4*(2**i)
  mesh_3d = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),ncell,ncell,ncell)
  radius  = 0.05
  dt      = 0.1#/ncell
  inv_dt  = Constant(1.0/dt)
  tpts    = int(T/dt)

  # === Perimeter and Area ===
  D_perimeter = 2*np.pi*radius
  D_area = np.pi*(radius**2)

  # === Create Meshes ===
  Omega = MeshFunction("size_t",mesh_3d,1,0)
  CompiledSubDomain("near(x[0],0.0) && near(x[1],0.0)").mark(Omega,1)
  Lambda = EmbeddedMesh(Omega,1)
  Lambda_boundaries = MeshFunction("size_t", Lambda, Lambda.topology().dim() - 1, 0)
  CompiledSubDomain("near(x[2], -0.5)").mark(Lambda_boundaries, 1)
  CompiledSubDomain("near(x[2],  0.5)").mark(Lambda_boundaries, 2)

  # === Function Spaces ===
  V3 = FunctionSpace(mesh_3d, "CG", 1)
  V1 = FunctionSpace(Lambda, "DG", 1)
  W = [V3, V1]
  u3, u1 = map(TrialFunction, W)
  v3, v1 = map(TestFunction, W)

  # === Velocity terms ===
  q1 = Constant(1.0)
  q3 = as_vector((0,0,q1))

  # === Initial concentration ===
  cinlet = Expression("(sin(pi*x[2])+2)*t", degree=4, t=0.0)#Constant(1.0)


  # === Mesh Measures ===
  dxOmega = Measure("dx", domain=mesh_3d)
  dxLambda = Measure("dx", domain=Lambda)
  dsLambda = Measure("ds", domain=Lambda, subdomain_data=Lambda_boundaries)
  #dsLambda = Measure("ds", domain=Lambda)
  #N = FacetNormal(mesh_3d)
  n = FacetNormal(Lambda)
  h = CellDiameter(Lambda)
  alpha = Constant(50.0)
  epsilon = Constant(-1.0)
  perf1 = 1.0
  perf3 = 1.0

  # === Upwind ===
  #u1_up = u1('+')
  #v1_up = v1('-')
  #q3_con = Constant((0,0,q1))
  #q1_up = (dot(q3_con, n) + abs(dot(q3_con, n))) / 2.0
  q1_up = (q1 + abs(q1))/2.0 # works, gets okay convergence - rates are off

  # === Cylinder ===
  cylinder = Circle(radius=radius, degree=20)

  # === Averages ===
  u3_avg = Average(u3, Lambda, cylinder)
  v3_avg = Average(v3, Lambda, cylinder)

  # === Initialize time step ===
  uh3d_prev = interpolate(Expression('0.0',degree=1), V3)
  uh1d_prev = interpolate(Expression('0.0',degree=1), V1)

  r = Expression('sqrt(pow(x[0] - 0.0, 2)+ pow(x[1] - 0.0,2))', degree  = 7)

  # === Begin time-stepping ===
  t = 0.0
  for step in range(tpts):
    t += dt
    print(f"[INFO] Time step {step+1}/{tpts}, t = {t:.3}")

    cinlet.t = t
    # === Exact solutions ===

    u3_exact_expr = Expression('(r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*((sin(pi*x[2])+2)*t) : (kappa/(1+kappa))*((sin(pi*x[2])+2)*t))',
                              degree = 7,
                              radius = radius,
                              kappa = kappa ,
                              r = r,
                              t = t)
    u3_exact = interpolate(u3_exact_expr, V3)

    u1_exact_expr = Expression('(sin(pi*x[2])+2)*t', degree = 7, t=t)
    u1_exact = interpolate(u1_exact_expr, V1)

    vtkfile = File('u3_exact.pvd')
    vtkfile << u3_exact
    vtkfile = File('u1_exact.pvd')
    vtkfile << u1_exact

    # === Forcing terms ===
    f3 = Expression('(r > radius ? area*(kappa/(1+kappa))*(1-radius*std::log(r/radius))*(sin(pi*x[2])+2) + (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pow(pi,2)*sin(pi*x[2])*t + (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pi*t*q*cos(pi*x[2]) :(kappa/(1+kappa))*pow(pi,2)*sin(pi*x[2]))*t + area*(kappa/(1+kappa))*(sin(pi*x[2])+2 + (kappa/(1+kappa))*pi*t*q*cos(pi*x[2]))',
                    degree = 4,
                    kappa = kappa,
                    radius = radius,
                    area = D_area,
                    r = r,
                    t = t,
                    q = q1)

    f1 = Expression(
        'area * (sin(pi*x[2])+2)'                             # time derivative
        ' + area * pow(pi,2) * t * sin(pi*x[2])'               # diffusion
        ' + (kappa / (1 + kappa)) * perimeter * t * (sin(pi*x[2]) + 2)'  # coupling
        ' + area * pi * t * q * cos(pi*x[2])',                # advection
        degree=4,
        area=D_area,
        perimeter=D_perimeter,
        kappa=kappa,
        t=t,
        q=q1
    )

    # === Checks for force terms ===
    f3_func = interpolate(f3, V3)
    #f1_func = interpolate(f1, V1)
    #File(f"f3_step{step:03d}.pvd") << f3_func
    #File(f"f1_step{step:03d}.pvd") << f1_func

    # === Variational Forms ===
    a00 = D_area * inv_dt * (inner(u3, v3) * dxOmega) + inner(grad(u3), grad(v3)) * dxOmega \
          + D_perimeter * kappa * inner(u3_avg, v3_avg) * dxLambda + inner(dot(q3,grad(u3)),v3) *dxOmega
    a01 = -kappa * D_perimeter * inner(u1, v3_avg) * dxLambda
    a10 = -kappa * D_perimeter * inner(u3_avg, v1) * dxLambda
    a11 = D_area * inv_dt * u1 * v1 * dxLambda \
        + D_area * inner(grad(u1), grad(v1)) * dxLambda \
        - D_area * inner(avg(grad(u1)), jump(v1, n)) * dS \
        - D_area * inner(avg(grad(v1)), jump(u1, n)) * dS \
        + (alpha/avg(h)) * jump(u1) * jump(v1) * dS \
        + kappa * D_perimeter * u1 * v1 * dxLambda \
        - D_area * q1 * u1 * v1.dx(2) * dxLambda \
        + D_area * jump(v1) * (q1_up * u1('+') - q1_up * u1('-')) * dS + D_area * q1_up * u1 * v1 * dsLambda(2) #* q1 * u1_up * jump(v1) * dS



    L0 = inner(f3, v3) * dxOmega + D_area * inv_dt * (inner(uh3d_prev, v3) * dxOmega)
    L1 = inner(f1, v1) * dxLambda \
        + D_area * inv_dt * uh1d_prev * v1 * dxLambda \
        + D_area * q1_up * cinlet * v1 * dsLambda(1)

    tau_SUPG = h / (2.0 * sqrt(dot(q3, q3)) + 1e-10)  # or h instead of h_out[i] for element-wise
    a00 += tau_SUPG * dot(q3, grad(v3)) * (f3 - div(-grad(u3) + q3*u3)) * dxOmega

    a = [[a00, a01], [a10, a11]]
    L = [L0, L1]

    #bc1d_inlet = DirichletBC(V1, u1_exact_expr, Lambda_boundaries, 1)
    bc3d = DirichletBC(V3, u3_exact, "on_boundary")
    W_bcs = [bc3d, []]

    A, b = map(ii_assemble, (a, L))
    A, b = apply_bc(A, b, W_bcs)
    A, b = map(ii_convert, (A, b))

    solver = PETScKrylovSolver()
    solver.set_operators(A, A)
    ksp = solver.ksp()

    opts = PETSc.Options()
    opts.setValue('ksp_type', 'gmres')
    opts.setValue('ksp_norm_type', 'unpreconditioned')
    opts.setValue('ksp_atol', 1E-14)
    opts.setValue('ksp_rtol', 1E-30)
    opts.setValue('ksp_monitor_true_residual', None)
    opts.setValue('pc_type', 'hypre')
    ksp.setFromOptions()

    x = b.copy()
    solver.solve(A,x,b)

    wh = ii_Function(W)
    wh.vector()[:] = x
    uh3d, uh1d = wh

    uh3d_prev.assign(uh3d)
    uh1d_prev.assign(uh1d)

    avg_uh3d = Average(uh3d, Lambda, cylinder)
    avg_u3d_exact = Average(u3_exact, Lambda, cylinder)

    # === Store Model Predictions ===
    vtkfile = File('3d_Prediction.pvd')
    vtkfile << uh3d

    vtkfile = File('1d_Prediction.pvd')
    vtkfile << uh1d


  # === Computer Errors ===
  h_out[i] =uh3d.function_space().mesh().hmax()
  H1_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'H1')
  L2_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'L2')
  err_interface[i] = 1.0
  H1_rates_3d[i] = ln(H1_error_3d[i]/H1_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_3d[i] = ln(L2_error_3d[i]/L2_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  rates_interface[i] = ln(err_interface[i]/err_interface[i-1])/ln(h_out[i]/h_out[i-1])

  H1_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'H1')
  L2_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'L2')
  H1_rates_1d[i] = ln(H1_error_1d[i]/H1_error_1d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_1d[i] = ln(L2_error_1d[i]/L2_error_1d[i-1])/ln(h_out[i]/h_out[i-1])

  print(i,"{:.3e}".format(h_out[i]),"{:.3e}".format(H1_error_3d[i]), "{:.3e}".format(H1_rates_3d[i]), "{:.3e}".format(err_interface[i]),"{:.3e}".format(rates_interface[i]), "{:.3e}".format(L2_error_3d[i]),"{:.3e}".format(L2_rates_3d[i]),  file= conv_study3d,flush = True)
  print(i,"{:.3e}".format(ncell),"{:.3e}".format(H1_error_1d[i]), "{:.3e}".format(H1_rates_1d[i]), "{:.3e}".format(L2_error_1d[i]),"{:.3e}".format(L2_rates_1d[i]),  file= conv_study1d,flush = True)





[INFO] Time step 1/10, t = 0.1


Averaging over 4 cells: 100%|██████████| 4/4 [00:00<00:00, 452.74it/s]


KeyError: Mesh(VectorElement(FiniteElement('Lagrange', Cell('interval', 3), 1), dim=3), 79367)

In [ ]:
# Print table like Table 7.1
print("\nTable 7.1-like Results")
print(f"{'N':>4} | {'||e3||_H1':>10} | {'Rate':>6} | {'||e3||_L2':>10} | {'Rate':>6} | {'||e1||_H1':>10} | {'Rate':>6} | {'||e1||_L2':>10} | {'Rate':>6}")
print("-"*90)
for i in range(len(mesh_vec)):
    row = f"{mesh_vec[i]:4d} | {H1_error_3d[i]:10.3e} |"
    row += f" {H1_rates_3d[i]:6.3f} |" if i > 0 else "        |"
    row += f" {L2_error_3d[i]:10.3e} |"
    row += f" {L2_rates_3d[i]:6.3f} |" if i > 0 else "        |"
    row += f" {H1_error_1d[i]:10.3e} |"
    row += f" {H1_rates_1d[i]:6.3f} |" if i > 0 else "        |"
    row += f" {L2_error_1d[i]:10.3e} |"
    row += f" {L2_rates_1d[i]:6.3f} |" if i > 0 else "        |"
    print(row)